# KERAS | EMBEDDING

In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding


Using TensorFlow backend.


In [2]:
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']


In [3]:
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])


In [4]:
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)


[[30, 33], [25, 14], [35, 28], [21, 14], [22], [28], [37, 28], [1, 25], [37, 14], [9, 12, 33, 27]]


In [5]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)


[[30 33  0  0]
 [25 14  0  0]
 [35 28  0  0]
 [21 14  0  0]
 [22  0  0  0]
 [28  0  0  0]
 [37 28  0  0]
 [ 1 25  0  0]
 [37 14  0  0]
 [ 9 12 33 27]]


In [6]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model

In [7]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


In [8]:
# summarize the model
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)


In [10]:
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 89.999998


---

# GLOVE

In [11]:
import os
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding


In [12]:
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']


In [13]:
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])


In [14]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1


In [15]:
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)


[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]


In [16]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)


[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


In [17]:
GLOVE_DIR = os.getcwd() + '\\glove.6B\\'

In [18]:
# load the whole embedding into memory


embeddings_index = dict()
#f = open('../glove_data/glove.6B/glove.6B.100d.txt')
f = open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt'), encoding="ISO-8859-1")
for line in f:
    values = line.split()
    word = values[0]
    if len(values[1:]) == 300:
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


Loaded 399883 word vectors.


In [19]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 300))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector


In [20]:
# define model
model = Sequential()
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))


In [21]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


In [22]:
# summarize the model
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 300)            4500      
_________________________________________________________________
flatten_2 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1201      
Total params: 5,701
Trainable params: 1,201
Non-trainable params: 4,500
_________________________________________________________________
None


In [23]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)


In [24]:
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


---